In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import statsmodels.api as sm
import plotly.express as px
import tqdm


from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

from google.colab import data_table
data_table.enable_dataframe_formatter()

# 0. Preprocessing of Dataset

In [4]:
from google.colab import files

# Open the file upload dialog
uploaded = files.upload()

# Get the file name
for file_name, data in uploaded.items():
    # Save the file to the local runtime
    with open(file_name, 'wb') as f:
        f.write(data)

Saving cucumbers.csv to cucumbers.csv


In [14]:
import pandas as pd

df= pd.read_csv(file_name)
import pandas as pd
import io

df = pd.read_csv(io.StringIO(uploaded[file_name].decode('utf-8')))

In [15]:
df_onehot = pd.get_dummies(df, columns=['country'])
df_onehot.drop(columns=["cycle"],inplace=True)
df_onehot.head(5)

,irrigation,sun,temperature,yield,country_Netherlands,country_Spain
0,2.657004,452.680020,20.450952,10.484136,0,1
1,4.361300,270.554441,22.211424,10.738837,0,1
2,3.930734,343.888517,20.479650,12.753241,0,1
3,3.912525,715.362181,15.710702,12.365033,0,1
4,2.636221,715.744196,14.974806,9.682015,0,1


In [16]:
df_onehot.dropna(inplace=True)
df_onehot.isnull().sum()
#rename countrys to include pesticide
df_onehot.rename(columns={"country_Netherlands":"Envidum_NED", "country_Spain": "Glyfanac_ESP"}, inplace=True)


In [17]:
#delete irrigation outliers
df_onehot = df_onehot.loc[df.loc[:, 'irrigation'] > 0]
df_onehot.describe()

,irrigation,sun,temperature,yield,Envidum_NED,Glyfanac_ESP
count,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000
mean,3.760319,393.754007,20.407213,10.876130,0.384779,0.615221
std,0.529158,177.051129,3.572012,1.489607,0.486673,0.486673
min,1.836051,86.560300,13.256176,3.930867,0.000000,0.000000
25%,3.410995,270.663868,17.742190,9.945715,0.000000,0.000000
50%,3.747738,370.457711,20.246858,11.053274,0.000000,1.000000
75%,4.117234,466.162155,22.930583,11.967669,1.000000,1.000000
max,5.419840,718.033027,33.795864,13.806458,1.000000,1.000000


In [18]:
#separate feature columns from target columns
target_columns = ['yield']
#dropping target columns from the set of columns gives the feature columns
feature_columns = df_onehot.columns.drop(target_columns)

#for convenience later, store number of features
n_features = len(feature_columns)

x = df_onehot.loc[:, feature_columns]
y = df_onehot.loc[:, target_columns]

y.head(5)

,yield
0,10.484136
1,10.738837
2,12.753241
3,12.365033
4,9.682015


In [19]:
#Train Test Split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

x_train

,irrigation,sun,temperature,Envidum_NED,Glyfanac_ESP
59,3.632819,428.188739,21.018146,1,0
599,3.830845,714.855212,19.942175,0,1
503,4.206823,343.510663,20.918559,0,1
1290,3.457380,466.378384,19.248115,0,1
1178,3.407927,467.281849,13.427445,0,1
...,...,...,...,...,...
1138,3.594899,271.419559,15.671746,0,1
1303,3.848513,371.380683,27.624511,1,0
866,3.502096,271.809435,22.572207,0,1
1468,3.930785,270.624743,18.911627,0,1


In [22]:
#Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [23]:
#Train Validation Split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

# 1. Create Neuronal Net

## 1.1 Basic Neuronal Net

In [24]:
neuralnet = Sequential()
neuralnet.add(Dense(89,activation="relu",use_bias=True,input_shape = (x_train.shape[1],)))
neuralnet.add(Dense(89,activation="relu",use_bias=True))
neuralnet.add(Dense(89,activation="relu",use_bias=True))
neuralnet.add(Dense(1,activation="relu",use_bias=True))
neuralnet.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mse'])

In [25]:
neuralnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 89)                534       
                                                                 
 dense_1 (Dense)             (None, 89)                8010      
                                                                 
 dense_2 (Dense)             (None, 89)                8010      
                                                                 
 dense_3 (Dense)             (None, 1)                 90        
                                                                 
Total params: 16,644
Trainable params: 16,644
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = neuralnet.fit(x_train,y_train, epochs=20, verbose = 1) 

Epoch 1/20
25/25 [==============================] - 3s 3ms/step - loss: 104.1717 - mse: 104.1717
Epoch 2/20
25/25 [==============================] - 0s 3ms/step - loss: 26.3625 - mse: 26.3625
Epoch 3/20
25/25 [==============================] - 0s 3ms/step - loss: 3.3595 - mse: 3.3595
Epoch 4/20
25/25 [==============================] - 0s 3ms/step - loss: 1.7242 - mse: 1.7242
Epoch 5/20
25/25 [==============================] - 0s 2ms/step - loss: 1.6179 - mse: 1.6179
Epoch 6/20
25/25 [==============================] - 0s 3ms/step - loss: 1.5463 - mse: 1.5463
Epoch 7/20
25/25 [==============================] - 0s 3ms/step - loss: 1.4899 - mse: 1.4899
Epoch 8/20
25/25 [==============================] - 0s 4ms/step - loss: 1.4406 - mse: 1.4406
Epoch 9/20
25/25 [==============================] - 0s 3ms/step - loss: 1.3799 - mse: 1.3799
Epoch 10/20
25/25 [==============================] - 0s 3ms/step - loss: 1.3222 - mse: 1.3222
Epoch 11/20
25/25 [==============================] - 0s 3ms/ste

In [27]:
Prediction_Neuralnet_Insample = neuralnet.predict(x_train)
Prediction_Neuralnet_Outofsample = neuralnet.predict(x_test)

15/15 [==============================] - 0s 1ms/step


In [28]:
print("In Sample:")
print("Neural Network: "+ str(mean_squared_error(y_train,Prediction_Neuralnet_Insample)))
print("Out of Sample:")
print("Neural Network: "+ str(mean_squared_error(y_test,Prediction_Neuralnet_Outofsample)))

In Sample:
Neural Network: 0.8190415118905238
Out of Sample:
Neural Network: 0.9337871766660384


In [29]:
neuralnet.evaluate(x_test,y_test)

15/15 [==============================] - 0s 2ms/step - loss: 0.9338 - mse: 0.9338


[0.9337871670722961, 0.9337871670722961]

## 1.2 Neuronal Net with Early Stopping

In [30]:
def create_model(num_layers,num_neurons,lr):
  model = Sequential()
  model.add(Dense(num_neurons,activation="relu",use_bias=True, input_shape = (x_train.shape[1],)))
  for j in range(num_layers-1):
    model.add(Dense(num_neurons,activation="relu",use_bias=True))
  model.add(Dense(1,activation="relu",use_bias=True))
  model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr),metrics=["mape"])
  return model

In [31]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor="val_loss",restore_best_weights=True, patience = 50)
es_network = create_model(num_layers= 3, num_neurons= 89, lr = 0.005)
es_network.fit(x_train,y_train, epochs=150, verbose = 1, validation_split=0.2, callbacks = [es])

Epoch 1/150
20/20 [==============================] - 1s 11ms/step - loss: 38.0173 - mape: 45.4641 - val_loss: 9.1978 - val_mape: 25.4158
Epoch 2/150
20/20 [==============================] - 0s 4ms/step - loss: 3.7217 - mape: 14.1631 - val_loss: 1.7309 - val_mape: 9.8931
Epoch 3/150
20/20 [==============================] - 0s 5ms/step - loss: 1.9243 - mape: 10.6753 - val_loss: 1.1972 - val_mape: 8.6598
Epoch 4/150
20/20 [==============================] - 0s 5ms/step - loss: 1.6391 - mape: 9.9268 - val_loss: 1.1788 - val_mape: 8.5240
Epoch 5/150
20/20 [==============================] - 0s 5ms/step - loss: 1.4622 - mape: 9.4552 - val_loss: 1.0547 - val_mape: 7.9314
Epoch 6/150
20/20 [==============================] - 0s 5ms/step - loss: 1.4136 - mape: 9.3383 - val_loss: 1.0042 - val_mape: 7.6943
Epoch 7/150
20/20 [==============================] - 0s 5ms/step - loss: 1.3254 - mape: 9.0572 - val_loss: 0.9864 - val_mape: 7.5582
Epoch 8/150
20/20 [==============================] - 0s 4ms/ste

In [32]:
es_network.evaluate(x_test,y_test)

15/15 [==============================] - 0s 3ms/step - loss: 0.0526 - mape: 1.7360


[0.052574947476387024, 1.736005187034607]

## 1.3 Neuronal Net with Hyperparameter search

In [35]:
import tensorflow as tf
from functools import partial
def create_model(num_layers,num_neurons,lr,droprate,l2_regularizer):
  model = Sequential()
  model.add(Dense(num_neurons,activation= "relu",use_bias=True, input_shape = (x_train.shape[1],)))
  model.add(Dropout(rate = droprate))
  for j in range(num_layers-1):
    model.add(Dense(num_neurons,activation= "relu",use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)))
    model.add(Dropout(rate = droprate))
  model.add(Dense(1,activation="relu",use_bias=True))
  model.compile(optimizer=Adam(learning_rate = lr),
              loss='mean_squared_error',  
              metrics=['mape'],
              ) 
  return model

In [36]:
# Hyperparameter
import random
np.random.seed(24)
num_models = 25
lr_or=0.002
l2 = 0.005
lr = np.random.uniform(lr_or/10,lr_or*10,num_models)
num_layers = np.random.randint(2,3,num_models)
num_neurons = np.random.randint(8,128,num_models)
l2_regularizer = np.random.uniform(l2/10,l2*10,num_models)
droprate = np.random.uniform(0.0,0.2,num_models)
d = {"lr":lr, "num_layers": num_layers, "num_neurons": num_neurons, "droprate": droprate, "l2_regularizer": l2_regularizer}
combinations = pd.DataFrame(d)
combinations.head(10)

,lr,num_layers,num_neurons,droprate,l2_regularizer
0,0.019208,2,35,0.007536,0.026472
1,0.014050,2,105,0.103762,0.016251
2,0.019997,2,41,0.020687,0.038357
3,0.004557,2,11,0.121534,0.013516
4,0.007349,2,12,0.112642,0.034375
5,0.014849,2,73,0.087938,0.023752
6,0.019930,2,18,0.056236,0.029457
7,0.006464,2,49,0.069558,0.030838
8,0.002904,2,64,0.198233,0.041244
9,0.007803,2,53,0.149276,0.008612


In [37]:
es = EarlyStopping(monitor="val_loss", patience=30,restore_best_weights=True)
val_loss = []
for j in tqdm.tqdm(range(num_models)):
  model_candidate = create_model(num_layers = combinations.loc[j,"num_layers"], num_neurons = combinations.loc[j,"num_neurons"],lr = combinations.loc[j,"lr"],droprate = combinations.loc[j,"droprate"],l2_regularizer = combinations.loc[j,"l2_regularizer"])
  history_candidate = model_candidate.fit(x_train, y_train, validation_split = 0.2, callbacks = [es],verbose=1, epochs =500)
  val_loss.append(np.min(history_candidate.history["val_loss"]))

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/500
20/20 [==============================] - 1s 23ms/step - loss: 35.1134 - mape: 43.8577 - val_loss: 11.0653 - val_mape: 26.8611
Epoch 2/500
20/20 [==============================] - 0s 9ms/step - loss: 5.0627 - mape: 15.6583 - val_loss: 2.6146 - val_mape: 10.6516
Epoch 3/500
20/20 [==============================] - 0s 13ms/step - loss: 2.8256 - mape: 11.6182 - val_loss: 1.9452 - val_mape: 9.2194
Epoch 4/500
20/20 [==============================] - 0s 9ms/step - loss: 2.4517 - mape: 10.7141 - val_loss: 1.8394 - val_mape: 8.9850
Epoch 5/500
20/20 [==============================] - 0s 7ms/step - loss: 2.2109 - mape: 10.2449 - val_loss: 1.6472 - val_mape: 8.5711
Epoch 6/500
20/20 [==============================] - 0s 8ms/step - loss: 2.1071 - mape: 10.1196 - val_loss: 1.6263 - val_mape: 8.7337
Epoch 7/500
20/20 [==============================] - 0s 5ms/step - loss: 1.9938 - mape: 9.9186 - val_loss: 1.5945 - val_mape: 8.7147
Epoch 8/500
20/20 [==============================] - 0s 4

  4%|▍         | 1/25 [00:28<11:18, 28.28s/it]

Epoch 1/500
20/20 [==============================] - 1s 12ms/step - loss: 30.8148 - mape: 40.5292 - val_loss: 2.8135 - val_mape: 10.5051
Epoch 2/500
20/20 [==============================] - 0s 5ms/step - loss: 5.3304 - mape: 16.0227 - val_loss: 3.5442 - val_mape: 12.7759
Epoch 3/500
20/20 [==============================] - 0s 5ms/step - loss: 3.8125 - mape: 13.3412 - val_loss: 1.8266 - val_mape: 8.6142
Epoch 4/500
20/20 [==============================] - 0s 4ms/step - loss: 2.8769 - mape: 11.6355 - val_loss: 1.6788 - val_mape: 8.2467
Epoch 5/500
20/20 [==============================] - 0s 5ms/step - loss: 2.7559 - mape: 11.4568 - val_loss: 1.5916 - val_mape: 8.0907
Epoch 6/500
20/20 [==============================] - 0s 5ms/step - loss: 2.6404 - mape: 11.3499 - val_loss: 1.4464 - val_mape: 7.7741
Epoch 7/500
20/20 [==============================] - 0s 4ms/step - loss: 2.4590 - mape: 11.0418 - val_loss: 1.3826 - val_mape: 7.7508
Epoch 8/500
20/20 [==============================] - 0s 5m

  8%|▊         | 2/25 [00:50<09:34, 25.00s/it]

...

 92%|█████████▏| 23/25 [11:31<01:08, 34.10s/it]

Epoch 1/500
20/20 [==============================] - 1s 12ms/step - loss: 41.4281 - mape: 47.1703 - val_loss: 10.9924 - val_mape: 26.6360
Epoch 2/500
20/20 [==============================] - 0s 5ms/step - loss: 5.3720 - mape: 16.0076 - val_loss: 2.4128 - val_mape: 9.4889
Epoch 3/500
20/20 [==============================] - 0s 5ms/step - loss: 3.0783 - mape: 11.3896 - val_loss: 1.9413 - val_mape: 8.6616
Epoch 4/500
20/20 [==============================] - 0s 5ms/step - loss: 2.7672 - mape: 10.8869 - val_loss: 1.7531 - val_mape: 8.2665
Epoch 5/500
20/20 [==============================] - 0s 6ms/step - loss: 2.5167 - mape: 10.6903 - val_loss: 1.6661 - val_mape: 7.9677
Epoch 6/500
20/20 [==============================] - 0s 5ms/step - loss: 2.2202 - mape: 9.8206 - val_loss: 1.5645 - val_mape: 7.8419
Epoch 7/500
20/20 [==============================] - 0s 6ms/step - loss: 2.2038 - mape: 10.0375 - val_loss: 1.5008 - val_mape: 7.7442
Epoch 8/500
20/20 [==============================] - 0s 5ms

 96%|█████████▌| 24/25 [12:10<00:35, 35.44s/it]

Epoch 1/500
20/20 [==============================] - 1s 12ms/step - loss: 32.6172 - mape: 41.1885 - val_loss: 5.2116 - val_mape: 14.5460
Epoch 2/500
20/20 [==============================] - 0s 5ms/step - loss: 6.6138 - mape: 17.6741 - val_loss: 2.3004 - val_mape: 8.7371
Epoch 3/500
20/20 [==============================] - 0s 5ms/step - loss: 5.4536 - mape: 16.0418 - val_loss: 1.9973 - val_mape: 8.5191
Epoch 4/500
20/20 [==============================] - 0s 6ms/step - loss: 5.1487 - mape: 15.4609 - val_loss: 1.8056 - val_mape: 8.2362
Epoch 5/500
20/20 [==============================] - 0s 6ms/step - loss: 4.2844 - mape: 14.6369 - val_loss: 1.6835 - val_mape: 7.9525
Epoch 6/500
20/20 [==============================] - 0s 5ms/step - loss: 3.6964 - mape: 13.3242 - val_loss: 1.7290 - val_mape: 8.7624
Epoch 7/500
20/20 [==============================] - 0s 6ms/step - loss: 3.9428 - mape: 14.3274 - val_loss: 1.4002 - val_mape: 7.6040
Epoch 8/500
20/20 [==============================] - 0s 5ms

100%|██████████| 25/25 [12:43<00:00, 30.52s/it]


In [38]:
idx_best_normal = np.argmin(val_loss)

In [39]:
es = EarlyStopping(monitor="val_loss",restore_best_weights=True, patience =200)
final_network = create_model(num_layers= combinations.loc[idx_best_normal,"num_layers"], num_neurons= combinations.loc[idx_best_normal,"num_neurons"], lr = combinations.loc[idx_best_normal,"lr"],droprate = combinations.loc[idx_best_normal,"droprate"],l2_regularizer = combinations.loc[idx_best_normal,"l2_regularizer"])
final_network.fit(x_train,y_train, verbose=1,validation_split=0.3,callbacks=[es], epochs=5000,batch_size=128)

Epoch 1/5000
5/5 [==============================] - 1s 43ms/step - loss: 101.8313 - mape: 90.0375 - val_loss: 44.6419 - val_mape: 59.8079
Epoch 2/5000
5/5 [==============================] - 0s 8ms/step - loss: 20.8387 - mape: 34.7821 - val_loss: 30.9661 - val_mape: 47.5721
Epoch 3/5000
5/5 [==============================] - 0s 11ms/step - loss: 19.3413 - mape: 34.0013 - val_loss: 2.9003 - val_mape: 11.5547
Epoch 4/5000
5/5 [==============================] - 0s 12ms/step - loss: 6.5613 - mape: 18.7834 - val_loss: 8.6636 - val_mape: 22.8012
Epoch 5/5000
5/5 [==============================] - 0s 8ms/step - loss: 7.1389 - mape: 19.9494 - val_loss: 2.7698 - val_mape: 11.3589
Epoch 6/5000
5/5 [==============================] - 0s 7ms/step - loss: 3.9513 - mape: 13.5386 - val_loss: 5.4241 - val_mape: 17.3602
Epoch 7/5000
5/5 [==============================] - 0s 8ms/step - loss: 3.8654 - mape: 13.3702 - val_loss: 2.5158 - val_mape: 10.7894
Epoch 8/5000
5/5 [==============================] - 0

In [40]:
print(final_network.evaluate(x_train,y_train))
print(final_network.evaluate(x_test,y_test))

25/25 [==============================] - 0s 2ms/step - loss: 0.0472 - mape: 1.3479
[0.047227174043655396, 1.3478527069091797]
15/15 [==============================] - 0s 2ms/step - loss: 0.0543 - mape: 1.5095
[0.054264962673187256, 1.5094563961029053]


In [44]:
final_network.save("/content/drive/My Drive/model_random_search.h5")

In [45]:
model = tf.keras.models.load_model("/content/drive/My Drive/model_random_search.h5")

In [46]:
model.weights

[<tf.Variable 'dense_83/kernel:0' shape=(5, 35) dtype=float32, numpy=
 array([[-3.30450326e-01, -8.43937546e-02, -2.86315233e-01,
         -3.62484872e-01,  1.19432651e-01, -1.61013201e-01,
          3.60617489e-02, -3.61347914e-01,  8.78025517e-02,
         -5.39230299e+00,  7.71905482e-02, -4.64784205e-02,
         -1.92232594e-01, -2.69596517e-01,  4.59102631e+00,
         -3.82052034e-01, -3.37738067e-01, -5.46865910e-03,
         -3.81291568e-01, -6.99826539e-01,  2.60509521e-01,
         -1.24069396e-02, -6.31170630e-01, -1.88623384e-01,
         -7.32011646e-02,  8.16302449e-02, -1.48100400e+00,
         -1.50468424e-02,  2.08418608e-01, -1.64256319e-02,
          1.05340295e-02, -4.07851070e-01, -1.94534570e-01,
         -3.19291621e-01, -3.11486274e-01],
        [ 2.68650949e-01, -2.24273931e-02, -1.39342332e-02,
          1.09296568e-01, -2.06944764e-01, -2.80395508e-01,
         -4.10195500e-01, -6.23610150e-03, -1.95363298e-01,
         -2.29336321e-02, -2.15377454e-02,  4.

In [47]:
model.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_83 (Dense)            (None, 35)                210       
                                                                 
 dropout_50 (Dropout)        (None, 35)                0         
                                                                 
 dense_84 (Dense)            (None, 35)                1260      
                                                                 
 dropout_51 (Dropout)        (None, 35)                0         
                                                                 
 dense_85 (Dense)            (None, 1)                 36        
                                                                 
Total params: 1,506
Trainable params: 1,506
Non-trainable params: 0
_________________________________________________________________


In [49]:
config = model.get_config()

In [50]:
config

{'name': 'sequential_27',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 5),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_83_input'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_83',
    'trainable': True,
    'batch_input_shape': (None, 5),
    'dtype': 'float32',
    'units': 35,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dropout',
   'config': {'name': 'dropout_50',
    'trainable': True,
    'dtype': 'float32',
    'rate': 0.007535746623292639,
    'noise_shape': None,
    'seed': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_84',
    'trainable': Tru